****

**Import The Necessary Libraries**

In [1]:
import numpy as np 
import pandas as pd 
import torch
import torch.nn as nn
from torchvision import transforms, datasets, models



import matplotlib.pyplot as plt

import imageio

from tqdm import tqdm


use transforms to augment data and make it so the resnet model will be able to work with our images(thay must be of size 224)

In [2]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
#     transforms.RandomRotation(degrees=(0, 180)),
#     transforms.CenterCrop(size=224),
    transforms.RandomHorizontalFlip(0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(size=256),
    transforms.CenterCrop(size=224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder('/kaggle/input/intel-image-classification/seg_train/seg_train', transform=train_transform)
test_dataset = datasets.ImageFolder('/kaggle/input/intel-image-classification/seg_test/seg_test', transform=test_transform)

load the data

In [3]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)


funcion defiend fo freezing and unfreezing our pre-trained model

In [4]:
def freeze_or_un_freeze(model ,val=False):
    for param in model.parameters():
        param.requires_grad = val

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')
print(device)

cuda:0


defining our training and accuracy methods

In [6]:
def accuracy(out, targets):
    _, preds = torch.max(out, dim=1)
    return torch.tensor(torch.sum(preds == targets).item() / len(preds))

def train(model, criterion, optimizer, train_loader, test_loader, epochs, unfreeze_after):
    train_losses = np.zeros(epochs)
    test_losses = np.zeros(epochs)
    train_accs = np.zeros(epochs)
    test_accs = np.zeros(epochs)
    
    for _ in tqdm(range(epochs)):
        
        if _ == unfreeze_after:
            freeze_or_un_freeze(model, True)
        
        train_loss = []
        train_acc = []
        test_acc = []
        
        model.train()
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            
            optimizer.zero_grad()
            
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
            train_acc.append(accuracy(outputs, targets))
        
        train_loss = np.mean(train_loss)
        train_acc = np.mean(train_acc)
        
        test_loss = []
        
        model.eval()
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_acc.append(accuracy(outputs, targets))

            test_loss.append(loss.item())
        test_loss = np.mean(test_loss)
        test_acc = np.mean(test_acc)
        
        train_losses[_] = train_loss
        test_losses[_] = test_loss
        train_accs[_] = train_acc
        test_accs[_] = test_acc
        
        print(f'Epoch {_+1}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')
    return train_losses, test_losses, train_accs, test_accs

In [82]:
model = models.resnet101(pretrained=True)
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [83]:
freeze_or_un_freeze(model)
n = model.fc.in_features
print(n)
model.fc = nn.Sequential(
    nn.Dropout(0.2),
    nn.Linear(n, 512), 
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, 6),
    nn.Softmax(),
)

2048


In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001, betas=(0, 0.999))


NameError: name 'model' is not defined

In [85]:
print(device)
model.to(device)
train(model, criterion, optimizer, train_loader, test_loader, 30, 10)

cuda:0


  3%|▎         | 1/30 [01:56<56:14, 116.37s/it]

Epoch 1, Train Loss: 1.6172, Test Loss: 1.3802, Train Acc: 0.6177, Test Acc: 0.8702


  7%|▋         | 2/30 [03:52<54:20, 116.46s/it]

Epoch 2, Train Loss: 1.3577, Test Loss: 1.2465, Train Acc: 0.8546, Test Acc: 0.8865


...

 97%|█████████▋| 29/30 [1:29:51<03:41, 221.69s/it]

Epoch 29, Train Loss: 1.0590, Test Loss: 1.1005, Train Acc: 0.9855, Test Acc: 0.9439


100%|██████████| 30/30 [1:33:32<00:00, 187.09s/it]

Epoch 30, Train Loss: 1.0593, Test Loss: 1.1022, Train Acc: 0.9857, Test Acc: 0.9416


(array([1.61717532, 1.35768946, 1.27359237, 1.2385327 , 1.21877538,
        1.20889992, 1.19702936, 1.19015058, 1.18506252, 1.17918133,
        1.13691294, 1.11158177, 1.10106334, 1.09189397, 1.086268  ,
        1.08385523, 1.07872464, 1.07425769, 1.07282956, 1.06966574,
        1.06991312, 1.066483  , 1.06606211, 1.06335202, 1.06202542,
        1.06063521, 1.06091288, 1.05944314, 1.05898595, 1.05925425]),
 array([1.38019052, 1.2465365 , 1.20815468, 1.19006454, 1.17972906,
        1.17374684, 1.16606487, 1.16080782, 1.15881382, 1.15790578,
        1.11502724, 1.10950824, 1.10686472, 1.10578779, 1.11566602,
        1.1045071 , 1.10909968, 1.10904689, 1.10660654, 1.10721463,
        1.10522882, 1.1035242 , 1.10633563, 1.10443056, 1.10415704,
        1.10346321, 1.10322707, 1.10644639, 1.10048554, 1.10222687]),
 array([0.61769962, 0.85457796, 0.86714602, 0.87506324, 0.87985641,
        0.87705642, 0.88552737, 0.88903922, 0.88666636, 0.89198941,
        0.91555148, 0.93660623, 0.94554383, 

In [87]:
train_losses=[]
test_losses=[]
def accuracy_final(loader, model):
    num_corrects = 0
    num_samples = 0
    model.eval()
    loop = tqdm(loader)
    with torch.no_grad():
        for x, y in loop:
            x = x.to(device)
            y = y.to(device)
            scores = model(x)
            test_losses.append(scores.data)
            _, prediction = scores.max(1)
            num_corrects += (prediction == y).sum()
            num_samples += prediction.size(0)
            acc = (num_corrects/num_samples) * 100
            loop.set_postfix(acc=acc.item())
        print(f'Got {num_corrects}/{num_samples} with accuracy {acc:.4f}')

accuracy_mehrdad(test_loader, model)

100%|██████████| 94/94 [00:18<00:00,  5.18it/s, acc=94.2]

Got 2825/3000 with accuracy 94.1667


lets try another aproach

In [13]:
model = models.resnet152(pretrained=True)
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [14]:
model = models.resnet152(pretrained=True)
print(model)
freeze_or_un_freeze(model)
n = model.fc.in_features
print(n)
model.fc = nn.Sequential(
    nn.Dropout(0.2),
    nn.Linear(n, 512), 
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, 6),
    nn.Softmax(),
)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001, betas=(0, 0.999))

2048


In [15]:
model.to(device)
train(model, criterion, optimizer, train_loader, test_loader, 30, 10)

  3%|▎         | 1/30 [03:27<1:40:03, 207.02s/it]

Epoch 1, Train Loss: 1.6215, Test Loss: 1.3869, Train Acc: 0.6249, Test Acc: 0.8756


  7%|▋         | 2/30 [05:49<1:18:53, 169.04s/it]

Epoch 2, Train Loss: 1.3674, Test Loss: 1.2564, Train Acc: 0.8492, Test Acc: 0.8789


 10%|█         | 3/30 [08:11<1:10:28, 156.60s/it]

...

 97%|█████████▋| 29/30 [1:59:41<04:59, 299.50s/it]

Epoch 29, Train Loss: 1.0589, Test Loss: 1.1058, Train Acc: 0.9858, Test Acc: 0.9366


100%|██████████| 30/30 [2:04:41<00:00, 249.37s/it]

Epoch 30, Train Loss: 1.0587, Test Loss: 1.1071, Train Acc: 0.9855, Test Acc: 0.9381


(array([1.62154042, 1.36742133, 1.28303813, 1.24429846, 1.22469136,
        1.20899252, 1.19890135, 1.19259588, 1.18652643, 1.18252647,
        1.14035904, 1.10974678, 1.09822216, 1.09121622, 1.08685227,
        1.08126099, 1.07449165, 1.07263337, 1.07144574, 1.06793341,
        1.06637014, 1.06471122, 1.06400162, 1.06236949, 1.06044867,
        1.06034315, 1.06081196, 1.05937214, 1.05887232, 1.05869326]),
 array([1.38694091, 1.25638575, 1.21290524, 1.1946052 , 1.18283801,
        1.17552867, 1.17157473, 1.16601787, 1.16201164, 1.15831631,
        1.11196652, 1.10633051, 1.10159806, 1.10636146, 1.10570289,
        1.10486785, 1.1062826 , 1.10418904, 1.10251463, 1.1032901 ,
        1.10529004, 1.10408998, 1.10651004, 1.10174267, 1.10064325,
        1.101247  , 1.11004928, 1.10328191, 1.10581246, 1.10712837]),
 array([0.62493676, 0.84924704, 0.86397433, 0.87490511, 0.87517399,
        0.8836292 , 0.88673753, 0.8891657 , 0.88907081, 0.88873863,
        0.91224533, 0.93758702, 0.94945902, 

okay this got worse so lets not use this aproach and lets stick to our first model

by the way sorry for not plotting the loss and accuracy on epoch i forgot to take the return and well now its too latr

In [ ]:
train(model, criterion, optimizer, train_loader, test_loader, 15, 5)

  7%|▋         | 1/15 [04:59<1:09:48, 299.18s/it]

Epoch 1, Train Loss: 1.0578, Test Loss: 1.1083, Train Acc: 0.9866, Test Acc: 0.9342


 13%|█▎        | 2/15 [09:58<1:04:54, 299.54s/it]

Epoch 2, Train Loss: 1.0571, Test Loss: 1.1038, Train Acc: 0.9873, Test Acc: 0.9387


 20%|██        | 3/15 [14:58<59:55, 299.65s/it]  

Epoch 3, Train Loss: 1.0561, Test Loss: 1.1032, Train Acc: 0.9881, Test Acc: 0.9419


 27%|██▋       | 4/15 [19:58<54:54, 299.51s/it]

Epoch 4, Train Loss: 1.0571, Test Loss: 1.1064, Train Acc: 0.9870, Test Acc: 0.9369


 33%|███▎      | 5/15 [24:57<49:55, 299.51s/it]

Epoch 5, Train Loss: 1.0573, Test Loss: 1.1106, Train Acc: 0.9867, Test Acc: 0.9328


In [ ]:
model2 = models.resnet34(pretrained=True)
print(model2)
freeze_or_un_freeze(model2)
n = model2.fc.in_features
print(n)
model2.fc = nn.Sequential(
    nn.Dropout(0.2),
    nn.Linear(n, 512), 
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, 6),
    nn.Softmax(),
)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model2.parameters(), lr=0.00001, betas=(0, 0.999))
train(model, criterion, optimizer, train_loader, test_loader, 30, 10)